<div style="background:#E9FFF6; color:#440404; padding:8px; border-radius: 4px; text-align: center; font-weight: 500;">IFN619 - Data Analytics for Strategic Decision Makers (2024 Sem 1)</div>

# IFN619 :: C1-UnstructuredAnalytics

For this session, the focus will be on analysis of unstructured text. However, the thinking required is similar to approaches to analysing images, video, sound and other unstructured data. Primarily, the analysis is based on the notion that there are useful patterns in the unstructured data which can be obtained mathematically. By converting the data to a mathematical structure, various algorithms can be applied to the structure with the aim of identifying patterns. 

In the case of the `topic modelling` approaches below, many of the techniques are *probabilistic* - that is they mathematically identify the *likelihood* that a feature might be important. Thus, they are never 100% accurate, and their use needs to be mediated by a more pragmatic *useful or not* approach, rather than *right or wrong*.

In [1]:
# Import the necessary libraries
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
import pandas as pd
import json
import random

### Accessing the data via The Guardian API

See the `Accessing_the_Guardian_API.ipynb` notebook file for details on getting the data. **Note:** This approach may be used for additional data for Assignment 2.

### Read in pre-saved data

To save time, we're loading in pre-saved data that was fetched using the Guardian API.

In [2]:
# Load the data - articles from The Guardian about the war in Ukraine
file_path = "data/"
file_name = "war_articles.json"

with open(f"{file_path}{file_name}",'r', encoding='utf-8') as fp:
    articles = json.load(fp)

print(f"Loaded {len(articles)} articles from {file_name}")

Loaded 84 articles from war_articles.json


Each dictionary entry includes the *title [date]* as `key` and the *body text* from the article as `value`.

In [3]:
article1 = list(articles.items())[0]
print("Key:",article1[0])
print("Value:",article1[1][:300],"...") # Just show first 300 characters

Key: Ukraine war briefing: France tells China – speak clearly to Russia about the war [2024-04-01T00:11:30Z]
Value:  France’s top diplomat has said China must speak clearly to Russia over its war in Ukraine. “We expect China to send very clear messages to Russia,” said Emmanuel Macron’s foreign minister, Stephane Sejourne, after meeting his Chinese counterpart, Wang Yi, in Beijing. “We are convinced that there wi ...


So the values gives us a list of documents that we can analyse.

In [4]:
# Get a list of documents
documents = list(articles.values())

# View first 400 characters of the 1st document
documents[0][:400]

' France’s top diplomat has said China must speak clearly to Russia over its war in Ukraine. “We expect China to send very clear messages to Russia,” said Emmanuel Macron’s foreign minister, Stephane Sejourne, after meeting his Chinese counterpart, Wang Yi, in Beijing. “We are convinced that there will be no lasting peace if it is not negotiated with the Ukrainians. “There will be no security for E'

### Term Count 

**Finding important terms by the frequency of their occurance**

Using `CountVectorizer` create a `vector` for each document where the dimensionality of the vector is the `vocabulary` (all terms in the collection), and the value of each component is the number of times that the `term` occurs in the document.

All of these analyses, approach the document as a [Bag of Words](https://en.wikipedia.org/wiki/Bag-of-words_model) model. In this approach, the order of the words don't matter. A popular approach that takes into account order is [Word embedding](https://en.wikipedia.org/wiki/Word_embedding). This session does not explore word embedding.

In [5]:
# Only count terms that in maximum of 75% of documents, and a minimum of 2 documents. 
# Count a maximum of 10000 terms, and remove common english stop words
count_vectorizer = CountVectorizer(max_df=0.75,min_df=2,max_features=10000,stop_words="english")
count_dt_matrix = count_vectorizer.fit_transform(articles.values())

In [6]:
# Take a look at the vector for the first document
doc001_vector = count_dt_matrix.toarray()[0]
doc001_vector

array([0, 0, 0, ..., 0, 0, 0])

In [7]:
# Get the 1000 terms identified during the vectorization process
feature_names = count_vectorizer.get_feature_names_out()
feature_names

array(['000', '00am', '00pm', ..., 'zone', 'zones', 'ישראל'], dtype=object)

In [8]:
# Look at how the counts match up to the terms (for the 1st doc)
doc001_term_counts = list(zip(feature_names,doc001_vector))
doc001_term_counts

[('000', 0),
 ('00am', 0),
 ('00pm', 0),
 ('01am', 0),
 ('01pm', 0),
 ('02am', 0),
 ('02pm', 0),
 ('037', 0),
 ('03am', 0),
 ('03pm', 0),
 ('049', 0),
 ('04am', 0),
 ('04pm', 0),
 ('05am', 0),
 ('05m', 0),
 ('05pm', 0),
 ('06am', 0),
 ('06pm', 0),
 ('07', 0),
 ('07am', 0),
 ('07pm', 0),
 ('08am', 0),
 ('08pm', 0),
 ('094', 0),
 ('09am', 0),
 ('09pm', 0),
 ('10', 0),
 ('100', 0),
 ('100bn', 0),
 ('100km', 0),
 ('100m', 0),
 ('100mm', 0),
 ('104', 0),
 ('107', 0),
 ('10am', 0),
 ('10km', 0),
 ('10m', 0),
 ('10pm', 0),
 ('11', 0),
 ('110', 0),
 ('111', 0),
 ('11am', 0),
 ('11c', 0),
 ('11pm', 0),
 ('12', 0),
 ('120', 0),
 ('125m', 0),
 ('129', 0),
 ('12am', 0),
 ('12pm', 0),
 ('13', 0),
 ('130', 0),
 ('130mm', 0),
 ('134', 0),
 ('13am', 0),
 ('13pm', 0),
 ('14', 1),
 ('140', 0),
 ('144', 0),
 ('14am', 0),
 ('14pm', 0),
 ('15', 0),
 ('150', 0),
 ('155', 0),
 ('15am', 0),
 ('15bn', 0),
 ('15pm', 0),
 ('16', 0),
 ('164', 0),
 ('16am', 0),
 ('16m', 0),
 ('16pm', 0),
 ('17', 0),
 ('170', 0),
 

In [9]:
# Take a look at the vocabulary which shows the total counts for whole collection
count_vectorizer.vocabulary_

{'france': 2985,
 'diplomat': 2184,
 'china': 1433,
 'speak': 6263,
 'clearly': 1483,
 'russia': 5850,
 'ukraine': 6968,
 'expect': 2676,
 'send': 5989,
 'clear': 1479,
 'messages': 4306,
 'emmanuel': 2485,
 'macron': 4136,
 'foreign': 2945,
 'stephane': 6381,
 'sejourne': 5978,
 'meeting': 4279,
 'chinese': 1434,
 'counterpart': 1840,
 'wang': 7219,
 'yi': 7420,
 'beijing': 1014,
 'convinced': 1797,
 'lasting': 3932,
 'peace': 4899,
 'negotiated': 4529,
 'ukrainians': 6970,
 'security': 5964,
 'europeans': 2606,
 'accordance': 404,
 'international': 3654,
 'law': 3946,
 'air': 552,
 'force': 2937,
 'shot': 6109,
 'launched': 3939,
 'shahed': 6044,
 'drones': 2362,
 'sunday': 6517,
 'night': 4558,
 'ukrainian': 6969,
 'military': 4343,
 'monday': 4408,
 'general': 3062,
 'staff': 6327,
 'did': 2167,
 'provide': 5271,
 'additional': 454,
 'details': 2130,
 'russian': 5851,
 'attacks': 856,
 'killed': 3853,
 'different': 2173,
 'regions': 5515,
 'eastern': 2410,
 'local': 4069,
 'officia

#### Display matrix in dataframe

Take the term count matrix and display in a dataframe to make visible the structure


In [10]:
# Create a new dataframe with the matrix - use titles for the index and terms for the columns
count_df = pd.DataFrame(count_dt_matrix.toarray(), index=articles.keys(), columns=feature_names)
count_df

,000,00am,00pm,01am,01pm,02am,02pm,037,03am,03pm,...,zaporizhzhia,zealand,zelenskiy,zero,zionist,znpp,zomi,zone,zones,ישראל
Ukraine war briefing: France tells China – speak clearly to Russia about the war [2024-04-01T00:11:30Z],0,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,0,0,0,0
Ukraine war briefing: Zelenskiy says troops have ‘stabilised’ battlefield positions [2024-04-05T01:24:08Z],1,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,0,0,0,0
Ukraine war briefing: alarm over drone attack on Zaporizhzhia power plant [2024-04-07T22:56:09Z],0,0,0,0,0,0,0,0,0,0,...,3,0,2,0,0,0,0,0,0,0
Ukraine war briefing: Russia warns France against deploying troops to Ukraine [2024-04-04T00:31:50Z],1,0,0,0,0,0,0,0,0,0,...,0,0,4,0,0,0,0,0,0,0
Ukraine war briefing: Kyiv electricity plant destroyed by Russian missiles and drones [2024-04-12T00:44:39Z],0,0,0,0,0,0,0,0,0,0,...,1,0,3,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Heavy rain continues in NSW; SA police head to Alice Springs for backup – as it happened [2024-04-05T06:50:15Z],4,1,0,2,1,1,1,0,1,0,...,0,0,0,0,0,0,6,0,2,0
Ten lawyers tell court Lehrmann may have leaked confidential material to Spotlight program – as it happened [2024-04-02T07:37:30Z],7,3,1,1,0,1,1,0,4,1,...,0,2,0,15,0,0,7,3,1,0
Teenager dead after stabbings – as it happened [2024-04-12T07:54:57Z],1,3,3,0,0,2,0,0,2,0,...,0,14,0,0,0,0,0,0,0,0
PM says transparency around aid worker’s death ‘in Israel’s interest’ – as it happened [2024-04-08T07:54:47Z],6,2,0,1,1,1,0,0,2,0,...,0,2,0,0,0,0,10,0,0,0


By selecting a row from the dataframe and sorting the values (counts), we can identify the top 10 terms

In [15]:
# Sample 5 random articles
samples = random.sample(range(0,len(count_df)),5)

for sample in samples:
    doc = count_df.iloc[sample]
    top_terms = dict(count_df.iloc[sample].sort_values(ascending=False).head(10))
    print(f"[{sample}] {doc.name}")
    print("\t- Top terms:",top_terms)

[65] Meet the Israeli Rabbi and survivor of 7 October attack now in Australia to campaign for peace [2024-04-01T14:00:09Z]
	- Top terms: {'israel': 9, 'says': 8, 'gaza': 6, 'human': 6, 'rights': 6, 'hamas': 5, 'believe': 5, 'australia': 5, 'october': 5, 'israelis': 4}
[1] Ukraine war briefing: Zelenskiy says troops have ‘stabilised’ battlefield positions [2024-04-05T01:24:08Z]
	- Top terms: {'russian': 16, 'region': 10, 'ukraine': 9, 'killed': 8, 'ukrainian': 8, 'nato': 7, 'attacks': 7, 'russia': 6, 'thursday': 6, 'eastern': 5}
[6] Ukraine war briefing: fresh hope for US House vote on Kyiv aid deal [2024-04-02T01:28:29Z]
	- Top terms: {'ukraine': 10, 'russia': 9, 'russian': 7, 'ukrainian': 6, 'monday': 5, 'moscow': 4, 'major': 3, 'country': 3, 'told': 3, 'sea': 2}
[27] Middle East crisis: Israel criticised for strike that killed seven aid workers in Gaza – as it happened [2024-04-02T18:28:13Z]
	- Top terms: {'gaza': 232, 'aid': 187, 'israeli': 139, 'israel': 137, 'workers': 130, 'bst':

#### Create a top10 terms dataframe

Using the index from the documents, create a dataframe that can hold the top10 terms for each document. We also include columns for our other analysis (tfidf, lda, nmf)

In [16]:
# Create a dataframe to hold top terms for each analysis type
terms_df = pd.DataFrame(index=count_df.index,columns=['count','tfidf','lda','nmf'])
terms_df

,count,tfidf,lda,nmf
Ukraine war briefing: France tells China – speak clearly to Russia about the war [2024-04-01T00:11:30Z],NaN,NaN,NaN,NaN
Ukraine war briefing: Zelenskiy says troops have ‘stabilised’ battlefield positions [2024-04-05T01:24:08Z],NaN,NaN,NaN,NaN
Ukraine war briefing: alarm over drone attack on Zaporizhzhia power plant [2024-04-07T22:56:09Z],NaN,NaN,NaN,NaN
Ukraine war briefing: Russia warns France against deploying troops to Ukraine [2024-04-04T00:31:50Z],NaN,NaN,NaN,NaN
Ukraine war briefing: Kyiv electricity plant destroyed by Russian missiles and drones [2024-04-12T00:44:39Z],NaN,NaN,NaN,NaN
...,...,...,...,...
Heavy rain continues in NSW; SA police head to Alice Springs for backup – as it happened [2024-04-05T06:50:15Z],NaN,NaN,NaN,NaN
Ten lawyers tell court Lehrmann may have leaked confidential material to Spotlight program – as it happened [2024-04-02T07:37:30Z],NaN,NaN,NaN,NaN
Teenager dead after stabbings – as it happened [2024-04-12T07:54:57Z],NaN,NaN,NaN,NaN
PM says transparency around aid worker’s death ‘in Israel’s interest’ – as it happened [2024-04-08T07:54:47Z],NaN,NaN,NaN,NaN


Populate the count column with data created by the count vectorizer.

In [17]:
#For each doc, get the 10 columns with the largest counts
for idx in terms_df.index:
    counts = dict(count_df.loc[idx].sort_values(ascending=False).head(10))
    #print(counts)
    terms_df.at[idx,'count'] = list(counts.keys()) # Just the list of terms

terms_df

,count,tfidf,lda,nmf
Ukraine war briefing: France tells China – speak clearly to Russia about the war [2024-04-01T00:11:30Z],"[russian, ukraine, regional, russia, region, s...",NaN,NaN,NaN
Ukraine war briefing: Zelenskiy says troops have ‘stabilised’ battlefield positions [2024-04-05T01:24:08Z],"[russian, region, ukraine, killed, ukrainian, ...",NaN,NaN,NaN
Ukraine war briefing: alarm over drone attack on Zaporizhzhia power plant [2024-04-07T22:56:09Z],"[ukraine, ukrainian, russia, russian, nuclear,...",NaN,NaN,NaN
Ukraine war briefing: Russia warns France against deploying troops to Ukraine [2024-04-04T00:31:50Z],"[ukraine, defence, president, russian, russia,...",NaN,NaN,NaN
Ukraine war briefing: Kyiv electricity plant destroyed by Russian missiles and drones [2024-04-12T00:44:39Z],"[ukraine, thursday, russian, plant, military, ...",NaN,NaN,NaN
...,...,...,...,...
Heavy rain continues in NSW; SA police head to Alice Springs for backup – as it happened [2024-04-05T06:50:15Z],"[bst, updated, south, weather, sydney, nsw, en...",NaN,NaN,NaN
Ten lawyers tell court Lehrmann may have leaked confidential material to Spotlight program – as it happened [2024-04-02T07:37:30Z],"[bst, updated, government, australia, says, au...",NaN,NaN,NaN
Teenager dead after stabbings – as it happened [2024-04-12T07:54:57Z],"[bst, australia, updated, government, dutton, ...",NaN,NaN,NaN
PM says transparency around aid worker’s death ‘in Israel’s interest’ – as it happened [2024-04-08T07:54:47Z],"[bst, updated, government, australia, australi...",NaN,NaN,NaN


### Term Frequency / Inverse Document Frequency (TF/IDF)

**Finding terms that are very common in a document, but less common in the whole collection**

The [TF/IDF](https://en.wikipedia.org/wiki/Tf–idf) algorithm takes the term frequencies for a document and divides them by the frequencies of the terms in the whole collection.


In [18]:
# Only count terms that in maximum of 75% of documents, and a minimum of 2 documents. 
# Count a maximum of 10000 terms, and remove common english stop words
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.75, min_df=2, max_features=10000, stop_words="english"
)

In [19]:
# Get the document vectors
tfidf_dt_matrix = tfidf_vectorizer.fit_transform(articles.values())

# Display the vector for the first document
tfidf_dt_matrix.toarray()[0]

array([0., 0., 0., ..., 0., 0., 0.])

#### Display matrix in dataframe

In [20]:
tfidf_df = pd.DataFrame(tfidf_dt_matrix.toarray(), index=articles.keys(), columns=tfidf_vectorizer.get_feature_names_out())
tfidf_df

,000,00am,00pm,01am,01pm,02am,02pm,037,03am,03pm,...,zaporizhzhia,zealand,zelenskiy,zero,zionist,znpp,zomi,zone,zones,ישראל
Ukraine war briefing: France tells China – speak clearly to Russia about the war [2024-04-01T00:11:30Z],0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.148367,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
Ukraine war briefing: Zelenskiy says troops have ‘stabilised’ battlefield positions [2024-04-05T01:24:08Z],0.018498,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.097722,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
Ukraine war briefing: alarm over drone attack on Zaporizhzhia power plant [2024-04-07T22:56:09Z],0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.141930,0.000000,0.091748,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
Ukraine war briefing: Russia warns France against deploying troops to Ukraine [2024-04-04T00:31:50Z],0.022567,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.158953,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
Ukraine war briefing: Kyiv electricity plant destroyed by Russian missiles and drones [2024-04-12T00:44:39Z],0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.043722,0.000000,0.127185,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Heavy rain continues in NSW; SA police head to Alice Springs for backup – as it happened [2024-04-05T06:50:15Z],0.013456,0.005754,0.000000,0.011847,0.006109,0.005195,0.006109,0.0,0.005320,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.025060,0.000000,0.013086,0.0
Ten lawyers tell court Lehrmann may have leaked confidential material to Spotlight program – as it happened [2024-04-02T07:37:30Z],0.017011,0.012471,0.004561,0.004279,0.000000,0.003753,0.004413,0.0,0.015373,0.004413,...,0.000000,0.009454,0.000000,0.066198,0.0,0.0,0.021121,0.012133,0.004727,0.0
Teenager dead after stabbings – as it happened [2024-04-12T07:54:57Z],0.003102,0.015920,0.017468,0.000000,0.000000,0.009581,0.000000,0.0,0.009812,0.000000,...,0.000000,0.084475,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
PM says transparency around aid worker’s death ‘in Israel’s interest’ – as it happened [2024-04-08T07:54:47Z],0.016818,0.009590,0.000000,0.004936,0.005090,0.004328,0.000000,0.0,0.008866,0.000000,...,0.000000,0.010904,0.000000,0.000000,0.0,0.0,0.034802,0.000000,0.000000,0.0


#### Update the terms matrix

In [21]:
for idx in terms_df.index:
    tfidf = dict(tfidf_df.loc[idx].sort_values(ascending=False).head(10))
    #print(counts)
    terms_df.at[idx,'tfidf'] = list(tfidf.keys()) 

terms_df

,count,tfidf,lda,nmf
Ukraine war briefing: France tells China – speak clearly to Russia about the war [2024-04-01T00:11:30Z],"[russian, ukraine, regional, russia, region, s...","[russian, russia, ukraine, regional, ukrainian...",NaN,NaN
Ukraine war briefing: Zelenskiy says troops have ‘stabilised’ battlefield positions [2024-04-05T01:24:08Z],"[russian, region, ukraine, killed, ukrainian, ...","[russian, nato, ukrainian, ukraine, region, ru...",NaN,NaN
Ukraine war briefing: alarm over drone attack on Zaporizhzhia power plant [2024-04-07T22:56:09Z],"[ukraine, ukrainian, russia, russian, nuclear,...","[ukrainian, ukraine, russia, russian, plant, n...",NaN,NaN
Ukraine war briefing: Russia warns France against deploying troops to Ukraine [2024-04-04T00:31:50Z],"[ukraine, defence, president, russian, russia,...","[ukraine, french, russian, russia, defence, na...",NaN,NaN
Ukraine war briefing: Kyiv electricity plant destroyed by Russian missiles and drones [2024-04-12T00:44:39Z],"[ukraine, thursday, russian, plant, military, ...","[ukraine, thursday, latvia, plant, manpower, r...",NaN,NaN
...,...,...,...,...
Heavy rain continues in NSW; SA police head to Alice Springs for backup – as it happened [2024-04-05T06:50:15Z],"[bst, updated, south, weather, sydney, nsw, en...","[bst, weather, nsw, updated, sydney, coast, so...",NaN,NaN
Ten lawyers tell court Lehrmann may have leaked confidential material to Spotlight program – as it happened [2024-04-02T07:37:30Z],"[bst, updated, government, australia, says, au...","[bst, updated, government, australia, says, co...",NaN,NaN
Teenager dead after stabbings – as it happened [2024-04-12T07:54:57Z],"[bst, australia, updated, government, dutton, ...","[bst, updated, arthur, dutton, australia, poli...",NaN,NaN
PM says transparency around aid worker’s death ‘in Israel’s interest’ – as it happened [2024-04-08T07:54:47Z],"[bst, updated, government, australia, australi...","[bst, updated, emerson, code, supermarkets, su...",NaN,NaN


#### Compare approaches

In [23]:
# Sample 5 random articles
samples = random.sample(range(0,len(terms_df)),5)

for sample in samples:
    doc = terms_df.iloc[sample]
    print(f"[{sample}] {doc.name}")
    print("\t- Counts:\t",doc['count'])
    print("\t- TFIDF:\t",doc['tfidf'])
    print()

[28] Penny Wong says Netanyahu trying to ‘brush aside’ death of Zomi Frankcom ‘deeply insensitive’ [2024-04-05T03:40:19Z]
	- Counts:	 ['wong', 'israeli', 'lerner', 'idf', 'australian', 'aid', 'investigation', 'israel', 'australia', 'netanyahu']
	- TFIDF:	 ['lerner', 'wong', 'israeli', 'idf', 'accept', 'insensitive', 'individual', 'rules', 'netanyahu', 'investigation']

[61] Meet the Sydney volunteers who are feeding the families fleeing Gaza [2024-04-03T13:51:34Z]
	- Counts:	 ['gaza', 'says', 'support', 'families', 'kitchen', 'boxes', 'provide', 'australia', 'visas', 'visitor']
	- TFIDF:	 ['gaza', 'families', 'says', 'visas', 'visitor', 'boxes', 'arrivals', 'support', 'repurposed', 'bake']

[73] Sally Capp looks back on six years as Melbourne lord mayor [2024-04-06T20:00:10Z]
	- Counts:	 ['city', 'says', 'melbourne', 'mayor', 'council', 'won', 've', 'business', 'initiatives', 'criticism']
	- TFIDF:	 ['city', 'melbourne', 'mayor', 'says', 'cbd', 'initiatives', 'won', 'pandemic', 'counci

### Topic modelling with Latent Dirichlet Allocation (LDA)

[LDA](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) is an algorithm for obtaining *topics* (a list of terms) from a document-term matrix. It is a generative probabilistic approach to *decomposition* of the document-term matrix into 2 factor matrices: document-topic and topic-term.

![img](https://editor.analyticsvidhya.com/uploads/26864dtm.JPG)

*Source: [Analytics Vidhya](https://www.analyticsvidhya.com/blog/2021/06/part-2-topic-modeling-and-latent-dirichlet-allocation-lda-using-gensim-and-sklearn/)*

The LDA model requires the number of topics to be set in advance. As it is a generative model, it also runs over a number of iterations. These values usually need to be experimented with to obtain quality topics.

In [24]:
# Set number of topics
num_topics = 20
# Set max number of iteractions
max_iterations = 10

# Create the model
lda_model = LatentDirichletAllocation(n_components=num_topics,max_iter=max_iterations,learning_method='online')

# Fit the model to the data, and use the model to transform the data (do the decomposition)
doc_topic_matrix = lda_model.fit_transform(count_dt_matrix)

# Obtain the topics
topic_term_matrix = lda_model.components_

#### View the topics

In [25]:
# Get the topics and their terms
lda_topic_dict = {}
for index, topic in enumerate(topic_term_matrix):
    zipped = zip(feature_names, topic)
    top_terms=dict(sorted(zipped, key = lambda t: t[1], reverse=True)[:10])
    print(top_terms)
    top_terms_list= {key : round(top_terms[key], 4) for key in top_terms.keys()}
    lda_topic_dict[f"topic_{index}"] = top_terms_list

# Print the topics with their terms    
for k,v in lda_topic_dict.items():
    print(k)
    print(v)
    print()

{'gaza': 3.9634367104713575, 'israel': 3.3607355428418617, 'bst': 3.2880802286899926, 'updated': 2.076748152618227, 'government': 1.94742090516542, 'israeli': 1.9218859282239813, 'aid': 1.8097353136065621, 'says': 1.7108300601265778, 'killed': 1.6195395035012463, 'australia': 1.5620319816049852}
{'bst': 3.1472459519140226, 'israel': 2.411077687552704, 'gaza': 2.086961070950483, 'updated': 1.9623478334941262, 'aid': 1.9319642758371287, 'israeli': 1.7174886163277003, 'killed': 1.1629471231621413, 'government': 1.1520958174709683, 'state': 1.1374825819089138, 'australia': 1.099604624614813}
{'bst': 1.9746430410992208, 'australia': 1.2656394095879906, 'government': 1.1144517457919847, 'updated': 0.9291265735135894, 'says': 0.917705135233178, 'gaza': 0.8803083474118072, 'state': 0.8084153479132122, 'israel': 0.80831374569966, 'aid': 0.7488018821878172, 'australian': 0.7425508524879639}
{'bst': 3.3508140573323306, 'israel': 3.2357130537063874, 'gaza': 2.8542209398543905, 'israeli': 2.6808225

#### List of topics for each document

In [30]:
doc_topic_matrix[0]

array([1.70068028e-04, 1.70068027e-04, 1.70068027e-04, 1.70068028e-04,
       1.70068028e-04, 1.70068028e-04, 1.70068028e-04, 1.70068028e-04,
       9.29717617e-01, 1.70068028e-04, 1.70068028e-04, 1.70068028e-04,
       6.72211581e-02, 1.70068027e-04, 1.70068027e-04, 1.70068027e-04,
       1.70068028e-04, 1.70068027e-04, 1.70068027e-04, 1.70068028e-04])

In [33]:
doc_topic_matrix[70]

array([9.82318274e-05, 9.82318274e-05, 9.82318274e-05, 9.82318274e-05,
       9.82318301e-05, 9.82318289e-05, 9.82318275e-05, 9.82318274e-05,
       9.82318285e-05, 9.82318274e-05, 9.82318277e-05, 9.82318413e-05,
       9.98133595e-01, 9.82318274e-05, 9.82318274e-05, 9.82318275e-05,
       9.82318279e-05, 9.82318274e-05, 9.82318274e-05, 9.82318281e-05])

#### Update the terms matrix

In [34]:
for idx,topic in enumerate(doc_topic_matrix):
    topic_num = topic.argmax()
    top_topic = lda_topic_dict[f"topic_{topic_num}"]
    terms_df['lda'].iloc[idx] = list(top_topic.keys())

terms_df

,count,tfidf,lda,nmf
Ukraine war briefing: France tells China – speak clearly to Russia about the war [2024-04-01T00:11:30Z],"[russian, ukraine, regional, russia, region, s...","[russian, russia, ukraine, regional, ukrainian...","[ukraine, russian, russia, ukrainian, region, ...",NaN
Ukraine war briefing: Zelenskiy says troops have ‘stabilised’ battlefield positions [2024-04-05T01:24:08Z],"[russian, region, ukraine, killed, ukrainian, ...","[russian, nato, ukrainian, ukraine, region, ru...","[ukraine, russian, russia, ukrainian, region, ...",NaN
Ukraine war briefing: alarm over drone attack on Zaporizhzhia power plant [2024-04-07T22:56:09Z],"[ukraine, ukrainian, russia, russian, nuclear,...","[ukrainian, ukraine, russia, russian, plant, n...","[ukraine, russian, russia, ukrainian, region, ...",NaN
Ukraine war briefing: Russia warns France against deploying troops to Ukraine [2024-04-04T00:31:50Z],"[ukraine, defence, president, russian, russia,...","[ukraine, french, russian, russia, defence, na...","[ukraine, russian, russia, ukrainian, region, ...",NaN
Ukraine war briefing: Kyiv electricity plant destroyed by Russian missiles and drones [2024-04-12T00:44:39Z],"[ukraine, thursday, russian, plant, military, ...","[ukraine, thursday, latvia, plant, manpower, r...","[ukraine, russian, russia, ukrainian, region, ...",NaN
...,...,...,...,...
Heavy rain continues in NSW; SA police head to Alice Springs for backup – as it happened [2024-04-05T06:50:15Z],"[bst, updated, south, weather, sydney, nsw, en...","[bst, weather, nsw, updated, sydney, coast, so...","[bst, israel, gaza, israeli, aid, updated, gov...",NaN
Ten lawyers tell court Lehrmann may have leaked confidential material to Spotlight program – as it happened [2024-04-02T07:37:30Z],"[bst, updated, government, australia, says, au...","[bst, updated, government, australia, says, co...","[bst, israel, gaza, israeli, aid, updated, gov...",NaN
Teenager dead after stabbings – as it happened [2024-04-12T07:54:57Z],"[bst, australia, updated, government, dutton, ...","[bst, updated, arthur, dutton, australia, poli...","[bst, israel, gaza, israeli, aid, updated, gov...",NaN
PM says transparency around aid worker’s death ‘in Israel’s interest’ – as it happened [2024-04-08T07:54:47Z],"[bst, updated, government, australia, australi...","[bst, updated, emerson, code, supermarkets, su...","[bst, israel, gaza, israeli, aid, updated, gov...",NaN


#### Compare approaches

In [35]:
# Sample 5 random articles
samples = random.sample(range(0,len(terms_df)),5)

for sample in samples:
    doc = terms_df.iloc[sample]
    print(f"[{sample}] {doc.name}")
    print("\t- Counts:\t",doc['count'])
    print("\t- TFIDF:\t",doc['tfidf'])
    print("\t- LDA:\t\t",doc['lda'])
    print()

[54] Morning Mail: voters back radical housing policies, ‘sports rorts’ documents revealed, total solar eclipse [2024-04-08T20:44:27Z]
	- Counts:	 ['housing', 'australia', 'public', 'support', 'morning', 'flies', 'day', 'country', 'poll', 'ukraine']
	- TFIDF:	 ['housing', 'fruit', 'flies', 'poll', 'solar', 'public', 'rugby', 'childcare', 'bridget', 'menopause']
	- LDA:		 ['bst', 'israel', 'gaza', 'israeli', 'aid', 'updated', 'government', 'australia', 'killed', 'workers']

[16] Fate of Middle East hangs in the balance as Israel mulls its next steps  [2024-04-14T04:53:20Z]
	- Counts:	 ['israel', 'netanyahu', 'attack', 'israeli', 'iranian', 'response', 'iran', 'biden', 'officials', 'defence']
	- TFIDF:	 ['israel', 'netanyahu', 'iranian', 'iran', 'israeli', 'response', 'attack', 'biden', 'gallant', 'officials']
	- LDA:		 ['bst', 'israel', 'gaza', 'israeli', 'aid', 'updated', 'government', 'australia', 'killed', 'workers']

[36] Anne Aly criticises Simon Birmingham for repeating Netanyahu’

### Topic modelling with Non-negative Matrix Factorisation (NMF)


[NMF](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) is a different algorithm for obtaining *topics* (a list of terms) from a document-term matrix. It also factorises the document-term matrix into 2 factor matrices: document-topic and topic-term.

In [36]:
# Set the number of topics
num_topics = 20

# Create the model
nmf_model = NMF(n_components=num_topics,init='random',beta_loss='frobenius')

# Fit the model to the data and use it to transform the data
doc_topic_nmf = nmf_model.fit_transform(tfidf_dt_matrix)

topic_term_nmf = nmf_model.components_

/opt/conda/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [37]:
# Get the topics and their terms
nmf_topic_dict = {}
for index, topic in enumerate(topic_term_nmf):
    zipped = zip(feature_names, topic)
    top_terms=dict(sorted(zipped, key = lambda t: t[1], reverse=True)[:10])
    #print(top_terms)
    top_terms_list= {key : round(top_terms[key], 4) for key in top_terms.keys()}
    nmf_topic_dict[f"topic_{index}"] = top_terms_list

# Print the topics with their terms    
for k,v in nmf_topic_dict.items():
    print(k)
    print(v)
    print()

topic_0
{'bst': 2.9085, 'updated': 1.3602, 'nsw': 0.4615, 'government': 0.4592, 'police': 0.3808, 'aap': 0.3537, 'australia': 0.3498, 'flooding': 0.348, 'south': 0.3368, 'weather': 0.3367}

topic_1
{'wong': 0.7158, 'lerner': 0.4804, 'israel': 0.4475, 'israeli': 0.4165, 'investigation': 0.3762, 'idf': 0.3571, 'aside': 0.3544, 'australian': 0.3283, 'workers': 0.3049, 'aid': 0.2929}

topic_2
{'mulch': 0.9196, 'heritage': 0.7719, 'roads': 0.7512, 'asbestos': 0.6437, 'council': 0.6195, 'concrete': 0.5618, 'melbourne': 0.4741, 'city': 0.4669, 'material': 0.3781, 'road': 0.3427}

topic_3
{'ukraine': 5.5101, 'russian': 5.398, 'russia': 4.0529, 'ukrainian': 2.8659, 'region': 1.5684, 'ministry': 1.3468, 'kharkiv': 1.3322, 'nato': 1.2706, 'zelenskiy': 1.2532, 'governor': 1.1805}

topic_4
{'crossword': 2.0496, 'notifications': 1.9541, 'australia': 1.7809, 'morning': 1.5736, 'world': 1.534, 'day': 1.5079, 'sydney': 1.4982, 'sign': 1.3582, 'sport': 1.3365, 'rugby': 1.2759}

topic_5
{'budget': 3.5414

#### Update the terms matrix

In [38]:
for idx,topic in enumerate(doc_topic_nmf):
    topic_num = topic.argmax()
    top_topic = lda_topic_dict[f"topic_{topic_num}"]
    terms_df['nmf'].iloc[idx] = list(top_topic.keys())

terms_df

,count,tfidf,lda,nmf
Ukraine war briefing: France tells China – speak clearly to Russia about the war [2024-04-01T00:11:30Z],"[russian, ukraine, regional, russia, region, s...","[russian, russia, ukraine, regional, ukrainian...","[ukraine, russian, russia, ukrainian, region, ...","[bst, israel, gaza, israeli, aid, military, ki..."
Ukraine war briefing: Zelenskiy says troops have ‘stabilised’ battlefield positions [2024-04-05T01:24:08Z],"[russian, region, ukraine, killed, ukrainian, ...","[russian, nato, ukrainian, ukraine, region, ru...","[ukraine, russian, russia, ukrainian, region, ...","[bst, israel, gaza, israeli, aid, military, ki..."
Ukraine war briefing: alarm over drone attack on Zaporizhzhia power plant [2024-04-07T22:56:09Z],"[ukraine, ukrainian, russia, russian, nuclear,...","[ukrainian, ukraine, russia, russian, plant, n...","[ukraine, russian, russia, ukrainian, region, ...","[bst, updated, australia, government, israel, ..."
Ukraine war briefing: Russia warns France against deploying troops to Ukraine [2024-04-04T00:31:50Z],"[ukraine, defence, president, russian, russia,...","[ukraine, french, russian, russia, defence, na...","[ukraine, russian, russia, ukrainian, region, ...","[bst, israel, gaza, israeli, aid, military, ki..."
Ukraine war briefing: Kyiv electricity plant destroyed by Russian missiles and drones [2024-04-12T00:44:39Z],"[ukraine, thursday, russian, plant, military, ...","[ukraine, thursday, latvia, plant, manpower, r...","[ukraine, russian, russia, ukrainian, region, ...","[bst, updated, australia, government, israel, ..."
...,...,...,...,...
Heavy rain continues in NSW; SA police head to Alice Springs for backup – as it happened [2024-04-05T06:50:15Z],"[bst, updated, south, weather, sydney, nsw, en...","[bst, weather, nsw, updated, sydney, coast, so...","[bst, israel, gaza, israeli, aid, updated, gov...","[gaza, israel, bst, updated, government, israe..."
Ten lawyers tell court Lehrmann may have leaked confidential material to Spotlight program – as it happened [2024-04-02T07:37:30Z],"[bst, updated, government, australia, says, au...","[bst, updated, government, australia, says, co...","[bst, israel, gaza, israeli, aid, updated, gov...","[gaza, israel, bst, updated, government, israe..."
Teenager dead after stabbings – as it happened [2024-04-12T07:54:57Z],"[bst, australia, updated, government, dutton, ...","[bst, updated, arthur, dutton, australia, poli...","[bst, israel, gaza, israeli, aid, updated, gov...","[gaza, israel, bst, updated, government, israe..."
PM says transparency around aid worker’s death ‘in Israel’s interest’ – as it happened [2024-04-08T07:54:47Z],"[bst, updated, government, australia, australi...","[bst, updated, emerson, code, supermarkets, su...","[bst, israel, gaza, israeli, aid, updated, gov...","[gaza, israel, bst, updated, government, israe..."


### Compare approaches

In [40]:
# Sample 5 random articles
samples = random.sample(range(0,len(terms_df)),5)

for sample in samples:
    doc = terms_df.iloc[sample]
    print(f"[{sample}] {doc.name}")
    print("\t- Counts:\t",doc['count'])
    print("\t- TFIDF:\t",doc['tfidf'])
    print("\t- LDA:\t\t",doc['lda'])
    print("\t- NMF:\t\t",doc['nmf'])
    print()

[0] Ukraine war briefing: France tells China – speak clearly to Russia about the war [2024-04-01T00:11:30Z]
	- Counts:	 ['russian', 'ukraine', 'regional', 'russia', 'region', 'sunday', 'attacks', 'ukrainian', 'military', 'killed']
	- TFIDF:	 ['russian', 'russia', 'ukraine', 'regional', 'ukrainian', 'sunday', 'zelenskiy', 'region', 'lviv', 'frontlines']
	- LDA:		 ['ukraine', 'russian', 'russia', 'ukrainian', 'region', 'attack', 'military', 'killed', 'drone', 'ministry']
	- NMF:		 ['bst', 'israel', 'gaza', 'israeli', 'aid', 'military', 'killed', 'hamas', 'government', 'says']

[13] Ukraine war briefing: Three killed in second day of Russian attacks on Zaporizhzhia region, says governor [2024-04-09T01:27:44Z]
	- Counts:	 ['russia', 'ukraine', 'military', 'attack', 'attacks', 'russian', 'moscow', 'foreign', 'nuclear', 'earlier']
	- TFIDF:	 ['russia', 'ukraine', 'russian', 'moscow', 'reactor', 'pass', 'zaporizhzhia', 'attacks', 'german', 'package']
	- LDA:		 ['ukraine', 'russian', 'russia',